
# CSCI 3143 - Lab 6: Stacks

**Name:** ______  

**Goals:**
1. To understand the abstract data types stack, queue, deque, and list.
2. To be able to implement the ADTs stack, queue, and deque using Python lists.
3. To understand the performance of the implementations of basic linear data structures.
4. To understand prefix, infix, and postfix expression formats.
5. To use stacks to evaluate postfix expressions.
6. To use stacks to convert expressions from infix to postfix.

**Reading:** Miller & Ranum, *Problem Solving with Algorithms and Data Structures using Python*, Ch. 3 (Sections 3.2–3.9).


## Linear Data Structures

Linear data structures organize data in a sequence where each element (except the first/last) has a predecessor and a successor. Typical linear structures you’ll see throughout the term:

- **Array / Python list (dynamic array)**
- **Linked list** (singly / doubly)
- **Stack**
- **Queue**
- **Deque** (double-ended queue)
- **Priority queue** (often implemented with a heap)
- **String** (conceptually linear, immutable in Python)

In this lab we focus on **stacks**, a last‑in/first‑out (LIFO) structure used for function call stacks, undo/redo, parsing expressions, and more.


## Using Generics and Type Variables

In Python, generics let us write flexible but type-safe code using the `typing` package. We create a type variable with TypeVar("T"), then define ADTs (like Stack[T] or Queue[T]) that can hold any type (int, str, bool, etc.) without rewriting the code. For example, Stack[int] enforces integers only, while Stack[str] enforces strings.


In [ ]:
from typing import List

# A list of ints
numbers: List[int] = [1, 2, 3, 4, 5]

# A list of bools
flags: List[bool] = [True, False, True]

# Type checkers (like mypy or Pyright) will enforce consistency:
numbers.append(6)  # ✅ ok
# numbers.append("oops") # ❌ type checker error

flags.append(False)  # ✅ ok
# flags.append(1)        # ❌ error: int is not bool

In [ ]:
from typing import TypeVar

T = TypeVar("T")


def identity(x: T) -> T:
    return x

This allows **parametric polymorphism**:
- Polymorphism = many forms.
- Parametric = parameterized by a type.

So parametric polymorphism means writing code that works for any type, with the type as a parameter. The behavior of the function/class doesn’t depend on what the type is; it works uniformly for all of them.

| Approach         | Example                  | Behavior                              | Safety |
|------------------|--------------------------|---------------------------------------|--------|
| No Type Hints    | `class Stack: ...`       | Allows any type, no checking          | ❌ None |
| `Any`            | `Stack[Any]`             | Explicitly allows mixing any types    | ❌ None |
| `TypeVar("T")`   | `Stack[T]`, e.g. `Stack[int]` | Enforces consistent element type | ✅ Strong |


### Generic Node Class

**Task 1:** Modify our Node class to take data of generic type, adding type annotations. Then create a node of type `int`.

Bonus: for variables that could be `None` or `Node[T]`, set as `Optional[Node[T]]`.

In [ ]:
from typing import TypeVar, Generic, Optional

T = TypeVar("T")


# --- Node Class
class Node(Generic[T]):
    def __init__(self, initdata):
        self.data = initdata
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self, newdata):
        self.data = newdata

    def setNext(self, newnext):
        self.next = newnext

### Generic Unordered List class

**Task 2:** Modify the following partial Unordered List class to have generic data type. Then create an unordered list of type `str`.

In [ ]:
class UnorderedList(Generic[T]):
    def __init__(self):
        self.head = None

    def isEmpty(self):
        return self.head == None

    def add(self, item):
        """Insert at head."""
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def size(self) -> int:
        current, count = self.head, 0
        while current is not None:
            count += 1
            current = current.getNext()
        return count

    def search(self, item) -> bool:
        current = self.head
        found = False
        while current is not None and not found:
            if current.getData() == item:
                found = True
            current = current.getNext()
        return found

    def to_list(self):
        the_list = list()
        current = self.head
        while current is not None:
            the_list.append(current.getData())
            current = current.getNext()
        return the_list

In [ ]:
myList = UnorderedList[str]()
myList.add("hello")
myList.add("there")
print(myList.to_list())

### Self Check

Check your code below:

In [ ]:
from typing import TypeVar, Generic, Optional, Self

T = TypeVar("T")


# --- Node Class
class Node(Generic[T]):
    def __init__(self, initdata: T) -> None:
        self.data: T = initdata
        self.next: Optional[Self] = None

    def getData(self) -> T:
        return self.data

    def getNext(self) -> Optional[Self]:
        return self.next

    def setData(self, newdata: T) -> None:
        self.data = newdata

    def setNext(self, newnext: Optional[Self]) -> None:
        self.next = newnext


# --- Unodered List Class
class UnorderedList(Generic[T]):
    def __init__(self):
        self.head = None

    def isEmpty(self) -> bool:
        return self.head == None

    def add(self, item: T) -> None:
        """Insert at head."""
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def size(self) -> int:
        current, count = self.head, 0
        while current is not None:
            count += 1
            current = current.getNext()
        return count

    def search(self, item: T) -> bool:
        current = self.head
        found = False
        while current is not None and not found:
            if current.getData() == item:
                found = True
            current = current.getNext()
        return found

    def to_list(self) -> list[T]:
        the_list = list[T]()
        current = self.head
        while current is not None:
            the_list.append(current.getData())
            current = current.getNext()
        return the_list

In [ ]:
myList = UnorderedList[str]()
myList.add("hello")
myList.add("there")
print(myList.to_list())


## The Stack Abstract Data Type (ADT)

A **stack** supports last in, first out (LIFO) access: the most recently added item is the first one removed. Core operations:

| Operation | Description | Complexity |
|---|---|---|
| `push(x)` | Add `x` to the top of the stack |  ?|
| `pop()` | Remove and return the top item | ? |
| `peek()` | Return (without removing) the top item | ? |
| `is_empty()` | Return `True` if stack has no items | ? |
| `size()` | Return the number of items | ? |


> Fill in the **Complexity** column as you complete the lab (consider the provided Python list‑backed implementation).



### Stack Implementation (List‑backed)

Below is a straightforward stack using Python’s dynamic array (`list`) as the underlying container.

In [ ]:
class Stack:
    def __init__(self):
        self._items = []

    def is_empty(self) -> bool:
        return len(self._items) == 0

    def push(self, item) -> None:
        self._items.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError("pop from empty stack")
        return self._items.pop()

    def peek(self):
        if self.is_empty():
            raise IndexError("peek from empty stack")
        return self._items[-1]

    def size(self) -> int:
        return len(self._items)

In [ ]:
# Quick sanity checks
s = Stack()
print(s.is_empty())
s.push(1)
s.push(2)
s.push(3)
print(s.size())
print(s.peek())
print(s.pop())
print(s.pop())
print(s.pop())
print(s.is_empty())
print("Stack basic checks passed.")

**Task 3:** Predict what the output of each of the following segments of code will be before testing our undersatnding

Predicted output:

In [ ]:
m = Stack()
m.push("x")
m.push("y")
m.pop()
m.push("z")
m.peek()

Predicted output:

In [ ]:
m = Stack()
m.push("x")
m.push("y")
m.push("z")
while not m.is_empty():
    m.pop()
    m.pop()
m.peek()

**Task 4:** Write a function rev_string(my_str) that uses a stack to reverse the characters in a string.

In [ ]:
def testEqual(got, expected):
    if got == expected:
        print(f"Pass: {got!r} == {expected!r}")
    else:
        print(f"Fail: {got!r} != {expected!r}")


def rev_string(my_str):
    # TODO: your code here
    raise NotImplementedError


testEqual(rev_string("apple"), "elppa")
testEqual(rev_string("x"), "x")
testEqual(rev_string("1234567890"), "0987654321")

### Stack Implementation with Linked List

**Task 5:** Modify our Stack class to use a linked list using your Node class. Then use this code to determine complexity for each method, completing the table at the beginning of this section.
*Optional challenge*: do so using generic type definitions.

In [ ]:
class Stack(Generic[T]):
    def __init__(self):
        self.head = None

    def is_empty(self) -> bool:
        return self.head is None

    def push(self, item: T) -> None:
        raise NotImplementedError

    def pop(self):
        if self.is_empty():
            raise IndexError("pop from empty stack")
        raise NotImplementedError

    def peek(self):
        if self.is_empty():
            raise IndexError("peek from empty stack")
        raise NotImplementedError

    def size(self) -> int:
        raise NotImplementedError

    
    def to_list(self) -> list[T]:
        the_list = list[T]()
        current = self.head
        while current is not None:
            the_list.append(current.getData())
            current = current.getNext()
        return the_list

In [ ]:
# checks
s = Stack()
print(s.is_empty())
s.push(1)
s.push(2)
s.push(3)
print(s.to_list())
print(s.size())
print(s.peek())
print(s.pop())
print(s.pop())
print(s.pop())
print(s.is_empty())
print("Stack basic checks passed.")

## Applications

### Balanced Parentheses Checker


Goal: To identify if a string has ballanced parenthesis.

Idea: Tcan the string left‑to‑right. Push `'('` onto the stack; on `')'` pop. If a close appears when empty, or you end nonempty, it’s unbalanced.

**Task 6:** Implement `parens_balanced(s: str) -> bool` to return whether a string containing only `'('` and `')'` is balanced.

**Examples:**  
- `""` → `True`  
- `"()"` → `True`  
- `"(()"` → `False`  
- `")("` → `False`


In [ ]:
def parens_balanced(s: str) -> bool:
    st = Stack()
    # TODO: Implement using Stack
    raise NotImplementedError


# Quick tests
tests = ["", "()", "(())", "(()", ")(", "())(", "((()))"]
for t in tests:
    print(t, " --> ", parens_balanced(t))


### Balanced Symbols (Multiple Types)

Extend the previous idea to handle `()`, `[]`, `{}`. When you see a closing symbol, it must match the most recent unmatched opener.


**Task 7:** Implement `symbols_balanced(s: str) -> bool` for parentheses, brackets, and braces. Ignore non‑bracket characters.

In [ ]:
def symbols_balanced(s: str) -> bool:
    # Use dictionary
    pairs = {")": "(", "]": "[", "}": "{"}
    opens = set(pairs.values())
    # TODO: Complete
    raise NotImplementedError


# Quick tests
cases = ["([]){}", "([)]", "((([{}])))", "][", "{[()]}[", "no symbols"]
for c in cases:
    print(c, " --> ", symbols_balanced(c))

### Decimal to Binary Conversion (Base 2)

Repeatedly divide by 2, pushing remainders. Then pop to produce the binary string (reverse of remainder order).

**Task 8.** Write `to_binary(n: int) -> str` that returns the base‑2 string for nonnegative `n`.

In [ ]:
def to_binary(n: int) -> str:
    # TODO: Complete
    raise NotImplementedError


for n in [0, 1, 2, 3, 7, 8, 13, 255]:
    print(n, "->", to_binary(n))


### Infix, Prefix, and Postfix

- **Infix:** operators between operands, e.g., `A + B`
- **Prefix (Polish):** operator before operands, e.g., `+ A B`
- **Postfix (RPN):** operator after operands, e.g., `A B +`

Stacks help convert and evaluate these because they easily track the “most recent” pending operator/operand.


**Algorithm sketch:**
1. Read tokens left→right.
2. Output operands immediately.
3. Push `'('` onto stack; on `')'`, pop to output until `'('`.
4. For an operator, pop and output any operators of **higher or equal** precedence from the stack first (respecting associativity), then push the new operator.


**Task 9.** Implement `infix_to_postfix(expr: str) -> str` for single‑letter/number operands and operators `+ - * / ^` with `^` right‑associative. Parentheses `()` supported.  
**Example:** `A * B + C * D` → `A B * C D * +`


In [ ]:
expr = "A * B + C * D"
print(expr.split())

In [ ]:
def infix_to_postfix(infix_expr):
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    op_stack = Stack()
    postfix_list = []
    token_list = infix_expr.split()

    for token in token_list:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfix_list.append(token)
        # TODO: else?

    raise NotImplementedError


tests = ["A * B + C * D", "( A + B ) * ( C + D )", "A + B * C"]
for t in tests:
    print(t, "->", infix_to_postfix(t))


**Task 10:** Answer the following:

1. Fill in the **Complexity** column for each stack operation using the provided list‑backed implementation. Explain briefly.
2. Why is a stack a good fit for balanced‑symbols checking?
3. In the infix→postfix algorithm, why is `^` treated as **right‑associative**?


---

If you run into issues with any section, add a short note describing where you got stuck and what you tried.


---

## Self‑Assessment
Please mark one option by editing the brackets to `[x]`:

- [ ] **10** – I completed all of this work on my own (learning from in‑class ideas/approaches).
- [ ] **8** – I completed most on my own, with some out‑of‑class help (peers/online).
- [ ] **6** – I needed significant help (peers/online/AI) to complete parts.
- [ ] **4** – I mostly copied code from others/AI and **do not** fully understand it.
- [ ] **2** – I copied almost everything without attempting to understand it.